<a href="https://colab.research.google.com/github/ignacioperezallub/ml_tensorflow_fcc/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

We first remove books that have less than 100 reviews, we have to do it before remove the users who submit less than 200 review.

In [ ]:
# groups by the "isbn" column and counts the number of rows for each 
# unique value of "isbn", the de dateFrame has the number of rated books by "isbn"
df_rating_isbn = df_ratings.groupby(["isbn"]).count().reset_index()

class_books = df_rating_isbn.loc[df_rating_isbn["rating"] >= 100]["isbn"]

# books contains those have no less than 100 ratings
class_books = df_books.loc[df_books["isbn"].isin(class_books)]



Now we made the same but for users that has at least 200 reviews

In [ ]:
df_rating_user = df_ratings[["user", "rating"]].groupby(["user"]).count().reset_index()

class_users = df_rating_user.loc[df_rating_user["rating"] >= 200]["user"]

df = df_ratings.loc[df_ratings["user"].isin(class_users)]
df = df.loc[df["isbn"].isin(class_books["isbn"])]

# Pivot the df to make, isbn > rows, users > columns , and ratings > values 
df_book_features = df.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)

# Convert dataframe of book features to scipy sparse matrix
sp_book_features = csr_matrix(df_book_features.values)


In [ ]:
model = NearestNeighbors(metric='cosine')
model.fit(sp_book_features)

NearestNeighbors(metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(title = ""):
  try:
        book = class_books.loc[class_books["title"] == title]
  except KeyError as e:
        print('Book', e, 'does not exist')
        return
  b = df_book_features.loc[df_book_features.index.isin(book["isbn"])]
  dist, index = model.kneighbors([x for x in b.values], n_neighbors=6)
  dist =  dist[0][1:]
  index = index[0][1:]

  titles = [ df_books.loc[df_books['isbn'] == df_book_features.iloc[i].name]["title"].values[0] for i in index]

  recommended = [list(z) for z in zip(titles, dist)][::-1]
  return [title, recommended]

get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
